## 1. Load the dataset
We will combine the  Description and Patient text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [9]:
# imports
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import time

In [10]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [11]:
# load & inspect dataset
df = pd.read_csv("../2-Data/dialogues.csv", sep = '\t')
df = df.dropna()#.head(1000)

In [12]:
df.rename(columns = {'Description':'Question',"Doctor":"Answer"}, inplace = True)

In [13]:
df

,Question,Patient,Answer
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...
...,...,...,...
256911,Why is hair fall increasing while using Bontre...,I am suffering from excessive hairfall. My doc...,"Hello Dear Thanks for writing to us, we are he..."
256912,Why was I asked to discontinue Androanagen whi...,"Hi Doctor, I have been having severe hair fall...","hello, hair4u is combination of minoxid..."
256913,Can Mintop 5% Lotion be used by women for seve...,Hi..i hav sever hair loss problem so consulted...,HI I have evaluated your query thoroughly you...
256914,Is Minoxin 5% lotion advisable instead of Foli...,"Hi, i am 25 year old girl, i am having massive...",Hello and Welcome to ‘Ask A Doctor’ service.I ...


In [14]:
df["combined"] = (
    "Question: " + df.Question.str.strip() + "; Patient: " + df.Patient.str.strip()+ "; Answer: " + df.Answer.str.strip()
)
df.head(2)

,Question,Patient,Answer,combined
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Question: Q. What does abutment of the nerve r...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Question: Q. What should I do to reduce my wei...


In [15]:
#df["combined"] = ( "Description: " + df.Description.str.strip() + "; Patient: " + df.Patient.str.strip())
#df.head(2)

In [16]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = df.shape[0]
#df = df.tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out

In [17]:
encoding = tiktoken.get_encoding(embedding_encoding)
# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

256916

In [10]:
df

,Description,Patient,Doctor,combined,n_tokens
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Description: Q. What does abutment of the nerv...,95
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Description: Q. What should I do to reduce my ...,519
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...,Description: Q. I have started to get lots of ...,285
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...,Description: Q. Why do I have uncomfortable fe...,324
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...,Description: Q. My symptoms after intercourse ...,442
...,...,...,...,...,...
256911,Why is hair fall increasing while using Bontre...,I am suffering from excessive hairfall. My doc...,"Hello Dear Thanks for writing to us, we are he...",Description: Why is hair fall increasing while...,211
256912,Why was I asked to discontinue Androanagen whi...,"Hi Doctor, I have been having severe hair fall...","hello, hair4u is combination of minoxid...",Description: Why was I asked to discontinue An...,154
256913,Can Mintop 5% Lotion be used by women for seve...,Hi..i hav sever hair loss problem so consulted...,HI I have evaluated your query thoroughly you...,Description: Can Mintop 5% Lotion be used by w...,191
256914,Is Minoxin 5% lotion advisable instead of Foli...,"Hi, i am 25 year old girl, i am having massive...",Hello and Welcome to ‘Ask A Doctor’ service.I ...,Description: Is Minoxin 5% lotion advisable in...,232


There are different ways to convert  text into a vector or into embeddings.

Unfortunately, most good methods to get embeddings in Python are not free.


## 2. Get embeddings using  SentenceTransformers

Let us use SentenceTransformers, a Python framework for state-of-the-art sentence, text, and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks.

First we verify that Torch is CUDA capable

In [18]:
import torch
torch.cuda.is_available()

True

We define our list of sentences. You can use a larger list (it is best to use a list of sentences for easier processing of each sentence)

We can install Sentence BERT using:
`!pip install sentence-transformers`



Step 1: We will then load the pre-trained BERT model. There are many other pre-trained models available.

In [19]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

We proceed to test the embeding creation

In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']
#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [21]:
sentence

['This framework generates embeddings for each input sentence']

In [22]:
def get_embeddings(x,transformer='paraphrase-MiniLM-L6-v2'):
    model = SentenceTransformer(transformer)
    #Sentences we want to encode
    sentence =x
    #Sentences are encoded by calling model.encode()
    embedding = model.encode(sentence)
    return embedding

In [23]:
# This may take a few minutes
embedding_mod='paraphrase-MiniLM-L6-v2'
#df["embedding"] = df.combined.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

In [24]:
df=df.head(1000)

In [26]:
#embedding_doctor
# This may take a few minutes
df["embedding"] = df.Answer.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

In [27]:
df

,Question,Patient,Answer,combined,n_tokens,embedding
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Question: Q. What does abutment of the nerve r...,95,"[-0.109211065, -0.17469415, 0.18996556, 0.0599..."
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Question: Q. What should I do to reduce my wei...,519,"[-0.014065318, 0.0440334, 0.26095688, 0.086799..."
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...,Question: Q. I have started to get lots of acn...,285,"[-0.39175138, -0.025890486, -0.024644196, -0.0..."
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...,Question: Q. Why do I have uncomfortable feeli...,324,"[-0.29406005, -0.31878802, 0.27588362, 0.09649..."
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...,Question: Q. My symptoms after intercourse thr...,442,"[-0.36187398, 0.18491694, -0.3090741, -0.30197..."
...,...,...,...,...,...,...
995,Q. My lax les is 38 cm with inflamed gastric f...,"Hello doctor, My lax les is 38 cm with inflame...",Hello. Gastritis is an inflammation of stomach...,Question: Q. My lax les is 38 cm with inflamed...,214,"[-0.1555396, -0.44157797, -0.15364785, 0.25760..."
996,Q. I am suffering from mood swings. Kindly adv...,"Hello doctor,I want to get some information re...",Hello. Let me answer your questions via some b...,Question: Q. I am suffering from mood swings. ...,491,"[-0.2296337, 0.119730674, 0.37153018, 0.062901..."
997,Q. I am having swollen lymph node in my neck. ...,"Hello doctor, I went to the chiropractor and g...",Hello. I do not think that because of chiropra...,Question: Q. I am having swollen lymph node in...,395,"[-0.10149522, -0.33532476, 0.40812746, -0.2713..."
998,Q. How good is Albenza for a raccoon roundworm...,"Hello doctor,I am concerned about a possible r...",Hello. Albendazole 400 mg single star dose is ...,Question: Q. How good is Albenza for a raccoon...,240,"[-0.06408733, 0.17669381, 0.09132431, -0.09456..."


In [28]:
from ast import literal_eval
import numpy as np

In [29]:
df["embedding"] = df.embedding.apply(np.array)  # convert string to numpy array

In [30]:
#df["embedding_doctor"] = df.embedding_doctor.apply(np.array)  # convert string to numpy array

In [31]:
df.to_pickle("../2-Data/dialogues_embededd.pkl")

In [32]:
#df.to_csv("../2-Data/dialogues_embededd.csv", sep = '\t', encoding='utf-8', index=False)

## 3. Get embeddings using OpenAI (optional)
If we have a subscription in OpenAI, you can follow the following steps.
Is optional, we are going to use the previous method.

In [24]:
# Python program to read
# json file
import json
# Opening JSON file
f = open('./credentials/api.json')
# returns JSON object as
# a dictionary
data = json.load(f)

In [28]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage
import openai
openai.api_key = data['OPENAI_API_KEY']
# Closing file
f.close()

In [42]:
# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [ ]:
df.to_csv("../2-Data/dialogues_embededd_openai.csv", sep='\t', encoding='utf-8', index=False)

## Additional Notes (not neeeded)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = ["I am doga.",
       "I am a dog"]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
# list of text documents
text = ["I am  doc.", "I am dog"]
# create the transform
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())